# 11049 : 행렬 곱셈 순서
- `n*m` 행렬과 `m*k` 행렬을 곱할 때 필요한 연산 수는 `n*m*k`이다. 필요한 최소 연산수를 구하자

In [26]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

n = int(input())

# 푸는 방식 자체는 유사해보임
# 근데 그걸 기억하느냐가 문제겠지
dp_matrix = [[0] * (n+1) for i in range(n+1)]
# acc_sum = [0] * (n+1)
lst = [0]

for i in range(n):
    lst.append(tuple(map(int, input().rstrip().split())))

# 누적합을 대체할 게 필요해보임 - 계산 순서가 바뀌면 연산이 달라지기 떄문임
# 누적합을 쓰는 이유는 누적합으로 부분합 구현이 가능했기 때문임
# 누적합으로 부분합을 구현할 수 있는가? - ㄴㄴ 
# 3*2*6 = 36인데 이건 누적합 리스트에서 구할수 없는 값임

# 앞에서 했던것과 마찬가지로 i는 길이, j는 시작점으로 놓고 반복문을 돌려보자
for i in range(2, n+1):
    for j in range(1, n-i+2):
        
        # 이 식을 잘 세우는 게 중요 : 근데 조금 다르게 가야 함 : 순서에 따라 곱해지는 값이 다르니까
        # 가령 [1][3]을 구한다고 하면 1 * [2][3] or [1][2] * 3 이 있을 건데
        # 앞에서 어떤게 곱해지느냐에 따라 뒤에서 곱해지는 케이스도 달라짐 
        
        dp_matrix[j][j+i-1] = min([dp_matrix[j][j+k] + dp_matrix[j+k+1][j+i-1] + (lst[j][0] * lst[j+k][1] * lst[j+i-1][1]) for k in range(i-1)])

print(dp_matrix[1][n])

[0, (5, 3), (3, 2), (2, 6), (6, 3)]
[0, 0, 0, 0, 0]
[0, 0, 30, 90, 96]
[0, 0, 0, 36, 54]
[0, 0, 0, 0, 36]
[0, 0, 0, 0, 0]
96


# 1520 : 내리막길
- 이동 가능한 경로의 수를 출력하시오

### 1트
- `RecursionError` : 그럴 것 같긴 했다

In [45]:
import sys
sys.stdin = open('test.txt', 'r')
sys.setrecursionlimit(10 ** 8) # 이거 있어도 시간초과 뜸
input = sys.stdin.readline

n, m = map(int, input().rstrip().split())
matrix = []
boundary = int(1e4 + 1)


matrix.append([boundary] * (m+2))
for i in range(n):
    matrix.append([boundary] + list(map(int, input().rstrip().split())) + [boundary])
matrix.append([boundary] * (m+2))
for i in matrix:
    print(i)
now = (0, 0)
count = 0
visited = []
# 경우의 수를 출력해야 하니까 재귀식으로 만들어야 할 것
# now 좌표가 (n, m)이면 count += 1 하면 될 것

def find_end(y, x):
    
    global count
    
    if y == n  and x == m:
        count += 1
        return
    
    else:
        print(y, x)

        # elif로 받으면 안됨!
        
        if matrix[y+1][x] < matrix[y][x]:
            find_end(y+1, x)
            
        if matrix[y][x+1] < matrix[y][x]:
            find_end(y, x+1)
        
        if matrix[y-1][x] < matrix[y][x]:
            find_end(y-1, x)

        if matrix[y][x-1] < matrix[y][x]:
            find_end(y, x-1)
            
        return

find_end(1, 1)
print(count)

[10001, 10001, 10001, 10001, 10001, 10001, 10001]
[10001, 50, 45, 37, 32, 30, 10001]
[10001, 35, 50, 40, 20, 25, 10001]
[10001, 30, 30, 25, 17, 28, 10001]
[10001, 27, 24, 22, 15, 10, 10001]
[10001, 10001, 10001, 10001, 10001, 10001, 10001]
1 1
2 1
3 1
4 1
4 2
4 3
4 4
1 2
1 3
1 4
2 4
3 4
4 4
1 5
2 5
2 4
3 4
4 4
3


### 조금 찾아보니까 dp를 이용할 수 있는데 대충 원리는 이렇다
- 방문한 적이 있다면 저장해두는 방식인데, 직접 구현해보자

In [73]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

n, m = map(int, input().rstrip().split())
matrix = []
for i in range(n):
    matrix.append(list(map(int, input().rstrip().split())))
dp = [[-1]*m for _ in range(n)]

# 1. 방문한 적 없다 : -1
# 2. 방문 중이다 : 0
# 3. 도착점까지 가는 경우의 수가 있다(dp[n-1][m-1]에 도달하면 1을 반환)
moves = [[1,0], [-1,0], [0,1], [0,-1]]
def find_end(x, y):
    
    if x == n-1 and y == m-1:
        return 1
    
    # 만난 적 있는 경로라면 굳이 계산을 더 하지 않아도 됨
    # dp의 각 점의 값은 그 점에서 도착점까지 도착하는 경우의 수임
    # 지나간 적 있는 경로이기 때문에 +1을 해줌. 이게 Memoization이죠?
    if dp[x][y] != -1:
        return dp[x][y]
    
    # 지금 점의 dp값이 -1일 때만 아래 코드를 따라감
    
    dp[x][y] = 0
    
    for move in moves:
        dx, dy = x + move[0], y + move[1]
        
        if 0 <= dx < n and 0 <= dy < m:
            if matrix[dx][dy] < matrix[x][y]:
                dp[x][y] += find_end(dx, dy)
                
    return dp[x][y]

# 
print(find_end(0, 0))

3

In [72]:
for i in dp:
    print(i)

[3, 2, 2, 2, 1]
[1, -1, -1, 1, 1]
[1, -1, -1, 1, -1]
[1, 1, 1, 1, -1]


In [70]:
# 가로 크기 n, 세로크기 m임
# 그냥 유클리드 좌표로 생각하지 말고 가로(row)의 수를 x, 세로(col)의 수를 y라고 생각하자

## 10942번 : 팰린드롬?
- 뒤집어도 똑같은 문자열을 팰린드롬이라고 한다고 함
- 아이디어 : 시작 ~ 끝 범위가 주어졌다고 하면
    - 양 끝의 범위를 하나씩 줄여나가며 팰린드롬인지 아닌지 비교하면 된다고 함
    - 팰린드롬이 나온 경우 
        1. 양 끝의 문자가 다르다면 팰린드롬이 아님
        2. 양 끝의 문자가 같다면 팰린드롬이 됨
    - 특수한 경우
        1. 시작 = 끝 인덱스라면 팰린드롬
        2. 길이가 2일때 시작 글자 = 끝 글자라면 팰린드롬

In [103]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
n = int(input())
lst = list(map(int, input().rstrip().split()))
m = int(input())

# 시간 초과
# for i in range(m):
#     s, e = map(int, input().rstrip().split())
#     if "".join(lst[s:e+1]) == "".join(lst[e:s-1:-1]):
#         print(1)
#     else:
#         print(0)

# DP를 생각해보자 - 2차원 리스트를 만들 수 밖에 없을 거 같은데
# 얘도 마찬가지로 길이 i와 시작점 j로 놓고 생각해보자
dp = [[0] * (n) for _ in range(n)]

for num_len in range(n):
    for start in range(n - num_len):
        end = start + num_len
        
        # 시작점과 끝점이 같다면 글자수가 1개이므로 무조건 팰린드롬
        if start == end:
            dp[start][end] = 1
            
        # 아하
        # 시작점의 글자와 끝점의 글자가 같다면
        elif lst[start] == lst[end]:
        	# 두 글자짜리 문자열이라면 무조건 팰린드롬
            if start + 1 == end: dp[start][end] = 1
            # 가운데 문자열이 팰린드롬이라면 팰린드롬
            elif dp[start+1][end-1] == 1: dp[start][end] = 1
            

#정답출력하기
for question in range(m):
    s, e = map(int, input().split())
    print(dp[s-1][e-1])



1
0
1
1


In [100]:
print(lst)
for i in dp:
    print(i)

[1, 2, 1, 3, 1, 2, 1]
[1, -1, -1, -1, -1, -1, -1]
[-1, 1, -1, -1, -1, -1, -1]
[-1, -1, 1, -1, -1, -1, -1]
[-1, -1, -1, 1, -1, -1, -1]
[-1, -1, -1, -1, 1, -1, -1]
[-1, -1, -1, -1, -1, 1, -1]
[-1, -1, -1, -1, -1, -1, 1]
